# Data Collection:

>> You have to scrape at least 1500 rows of data. You can scrape more data as well, it’s up to you, More the data better the model. In this section you have to scrape the data of flights from different websites (yatra.com, skyscanner.com, official websites of airlines etc). The number of columns for data doesn’t have limit, it’s up to you and your creativity. Generally, these columns are airline name, date of journey, source, destination, route, departure time, arrival time, duration, total stops and the target variable price. You can make changes to it, you can add or you can remove some columns, it completely depends on the website from which you are fetching the data.

In [21]:
# Importing the neccessary libraries
import pandas as pd
import time
import requests
from  bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,TimeoutException,ElementClickInterceptedException,StaleElementReferenceException,ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Scraping data from Yatra.com

In [22]:
#path for chromedriver
driver = webdriver.Chrome('chromedriver.exe')

# to open the webpage
url = "https://www.yatra.com/"
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [23]:
driver.find_element_by_xpath("//div[@class='ripple-parent search-height demo-icon icon-go']").click()
time.sleep(3)

In [24]:
#making list of locations
city = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [25]:
#empty lists to store data
airline_name=[]
depart_time = []
arrival_time = []
duration=[]
stops = []
source = []
destination = []
date=[]
meal=[]
price=[]
loc=[]

In [26]:
for x in city:
    for y in city:
        if x!=y:            
            source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            source.clear() 
            source.send_keys(x)
            time.sleep(2)
            
            destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            time.sleep(2)
            destination.click()
            destination.clear()
            time.sleep(2)
            destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(1)
            
            #scrolling the page
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(5)
            #clicking on view fare
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
           #fetching name of airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    airline_name.append(i.text)
            except NoSuchElementException:
                pass
            #time of departure
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    depart_time.append(i.text)
            except NoSuchElementException:
                pass
            #time of arrival
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            #duration
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    duration.append(i.text)
            except NoSuchElementException:
                pass
            #number of stops
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    stops.append(i.text)
            except NoSuchElementException:
                pass
            #fetching availability of meal
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    meal.append(i.text)
            except NoSuchElementException:
                pass
            #locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    loc.append(i.text)
            except NoSuchElementException:
                pass
            #fetching prices
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    price.append(i.text)
            except NoSuchElementException:
                pass

            

In [27]:
print(len(price),len(airline_name),len(depart_time),len(arrival_time),len(duration),len(stops),len(meal),len(price))

3445 3365 3445 3445 3445 3445 3445 3445


In [29]:
source = loc[0:len(loc):2]
destination = loc[1:len(loc):2]

In [30]:
print(len(source), len(destination))

3445 3445


In [31]:
# Creating the dataframe
df=pd.DataFrame({"Airline":airline_name,
                 "Dep_Time":depart_time[:3365],
                 "Arrival_Time":arrival_time[:3365],
                 "Duration":duration[:3365], 
                 "Source":source[:3365],
                 "Destination":destination[:3365],
                 "Meal_Availability":meal[:3365],
                 "Total_Stops":stops[:3365],
                 "Price":price[:3365]})
df

,Airline,Dep_Time,Arrival_Time,Duration,Source,Destination,Meal_Availability,Total_Stops,Price
0,Go First,08:00,10:10,2h 10m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
1,Go First,14:20,16:35,2h 15m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
2,Go First,21:00,23:15,2h 15m,New Delhi,Mumbai,eCash 250,Non Stop,"5,954"
3,Go First,16:45,21:25,4h 40m,New Delhi,Mumbai,eCash 250,1 Stop,"5,954"
4,Go First,12:35,18:10,5h 35m,New Delhi,Mumbai,eCash 250,1 Stop,"5,954"
...,...,...,...,...,...,...,...,...,...
3360,Air India,11:05,20:30,9h 25m,Lucknow,Goa,Free Meal,1 Stop,"12,084"
3361,Air India,08:50,14:55,6h 05m,Lucknow,Goa,eCash 250,1 Stop,"12,818"
3362,Air India,08:55,22:25,13h 30m,Lucknow,Goa,Free Meal,3 Stop(s),"13,137"
3363,Air India,08:55,06:25,21h 30m,Lucknow,Goa,Free Meal,2 Stop(s),"13,137"


In [32]:
#saving the data into excel file
df.to_excel("Flight_Price_Dataset.xlsx")